In [1]:
import os

In [2]:
%pwd

'f:\\zizo\\MLOPS\\datascienceproject\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\zizo\\MLOPS\\datascienceproject'

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )
        return data_transformation_config



In [7]:
import os
from src.datascience import logger
from sklearn.model_selection import train_test_split
import pandas as pd

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)
        
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)


In [ ]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2025-03-09 00:46:32,644: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-09 00:46:32,646: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-09 00:46:32,650: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-09 00:46:32,652: INFO: common: created directory at: artifacts]
[2025-03-09 00:46:32,653: INFO: common: created directory at: artifacts/data_transformation]
[2025-03-09 00:46:32,690: INFO: 1168814766: Splited data into training and test sets]
[2025-03-09 00:46:32,691: INFO: 1168814766: (1199, 12)]
[2025-03-09 00:46:32,692: INFO: 1168814766: (400, 12)]
(1199, 12)
(400, 12)
